In [1]:
# Sempre rode isso para começar o Notebook
# De modo resumido, isso faz com que o notebook possa acessar o Django

import sys
sys.path.append("../../app/")

import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "projeto.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from serase_app.models import *

In [3]:
# Digite seu código aqui!

In [4]:
# Testando values_list no PadraoMovimentacao

from serase_app.models import *
from django.db.models import F

lista = PadraoMovimentacao.objects.values("id", "descricao", "periodo", "dia_cobranca", "data_inicio", "data_fim", valor_padrao=F("valor"), categoria=F("cod_categoria__nome"), tipo=F("receita_despesa"))
lista = list(lista)
lista

FieldError: Cannot resolve keyword 'data_inicio' into field. Choices are: categoria, cod_categoria, cod_categoria_id, cod_usuario, cod_usuario_id, data_fim, data_geracao, descricao, dia_cobranca, divida, id, movimentacao, periodo, receita_despesa, tipo, valor, valor_padrao

In [4]:
# Testando o choices do PadraoMovimentacao

#from serase_app.models import PadraoMovimentacao
from django.db.models import F, Case, When, CharField, Value

# > Consulta

# Pego de https://stackoverflow.com/a/60158573
class WithChoices(Case):
    def __init__(self, model, field, condition=None, then=None, **lookups):
        choices = dict(model._meta.get_field(field).flatchoices)
        whens = [When(**{field: k, 'then': Value(v)}) for k, v in choices.items()]
        return super().__init__(*whens, output_field=CharField())


PadraoMovimentacao.objects.values()

<QuerySet [{'id': 1, 'receita_despesa': 'despesa', 'descricao': 'Spotify', 'periodo': '1', 'valor': Decimal('-26.90'), 'dia_cobranca': 1, 'data_inicio': datetime.date(2018, 4, 18), 'data_fim': None, 'cod_usuario_id': 1, 'cod_categoria_id': 4}, {'id': 2, 'receita_despesa': 'receita', 'descricao': 'Bolsa IC', 'periodo': '1', 'valor': Decimal('300.00'), 'dia_cobranca': 6, 'data_inicio': datetime.date(2020, 7, 8), 'data_fim': datetime.date(2021, 7, 8), 'cod_usuario_id': 1, 'cod_categoria_id': 9}, {'id': 3, 'receita_despesa': 'despesa', 'descricao': 'Conta de luz', 'periodo': '1', 'valor': None, 'dia_cobranca': 6, 'data_inicio': None, 'data_fim': None, 'cod_usuario_id': 1, 'cod_categoria_id': 10}, {'id': 4, 'receita_despesa': 'despesa', 'descricao': 'Internet', 'periodo': '1', 'valor': Decimal('-59.90'), 'dia_cobranca': 5, 'data_inicio': datetime.date(2020, 2, 4), 'data_fim': None, 'cod_usuario_id': 1, 'cod_categoria_id': 10}, {'id': 5, 'receita_despesa': 'receita', 'descricao': 'Sálario', 

In [11]:
# Testando movimentacoes

from serase_app.models import *

In [12]:
valores = Movimentacao.objects.filter(valor_pago__isnull=False).values_list("descricao", "data_lancamento").order_by("-data_lancamento")

for nom, dat in valores:
    print(nom + " " + str(dat))

Spotify 2021-01-17
Bolsa Ic 2021-01-08
Conta de luz 2021-01-08
internet 2021-01-08
TOP 2021-01-08
Sálario 2021-01-07
Flauta 2021-01-02
Mate coro e sanduba 2020-11-01
bebida Halloween 2020-10-30
Make pro Halloween 2020-10-30
Dinheiro do Spotify 2020-10-26
Quebrando a quarentena 2020-10-17
Compras 2020-10-10
Bolsa ic 2020-10-08
Conta de luz 2020-10-08
Internet 2020-10-08
Spotify 2020-10-08
Sálario 2020-10-07
Churros 2020-10-03
Milkshake McDonnadls 2020-09-15
Compras 2020-09-12
Bolsa ic 2020-09-08
Conta de luz 2020-09-08
Internet 2020-09-08
Internet 2020-09-08
Spotify 2020-09-08
Sálario 2020-09-07
Viola 2020-09-02
Bolsa ic 2020-08-10
Conta de luz 2020-08-10
Internet 2020-08-10
Spotify 2020-08-10
Sálario 2020-08-07
Adoção do Cumpa, o Cacto 2020-07-16
Outer wilds 2020-07-10


In [7]:
# Teste datetime: objeto data apartir de string e filtrar por data

from datetime import datetime

data = datetime.strptime("2020-10-26", '%Y-%m-%d').date()
Movimentacao.objects.filter(data_lancamento__gte=data)

<QuerySet [<Movimentacao: Mate coro e sanduba>, <Movimentacao: bebida Halloween>, <Movimentacao: Make pro Halloween>, <Movimentacao: Dinheiro do Spotify>, <Movimentacao: Bolsa Ic>, <Movimentacao: Conta de luz>, <Movimentacao: internet>, <Movimentacao: Sálario>, <Movimentacao: Flauta>, <Movimentacao: Spotify>]>

In [8]:
a = Movimentacao.objects.filter(descricao="Churros").values()
list(a)[0]

{'id': 13,
 'descricao': 'Churros',
 'valor_esperado': None,
 'valor_pago': Decimal('-5.00'),
 'data_geracao': None,
 'data_lancamento': datetime.date(2020, 10, 3),
 'cod_usuario_id': 1,
 'cod_categoria_id': 5,
 'cod_padrao_id': None}

In [9]:
from datetime import datetime

mes_ano = datetime.strptime("2020-10", '%Y-%m')

a = Movimentacao.objects.filter(data_lancamento__month=mes_ano.month, data_lancamento__year=mes_ano.year).order_by("-data_lancamento")
for q in a:
    print(q.descricao, q.data_lancamento)

bebida Halloween 2020-10-30
Make pro Halloween 2020-10-30
Dinheiro do Spotify 2020-10-26
Quebrando a quarentena 2020-10-17
Compras 2020-10-10
Bolsa ic 2020-10-08
Conta de luz 2020-10-08
Internet 2020-10-08
Spotify 2020-10-08
Sálario 2020-10-07
Churros 2020-10-03
mov_tests 2020-10-01


In [10]:
datetime.today().year

2021

In [11]:
from django.db.models import Sum

mes_ano = datetime.strptime("2020-10", '%Y-%m')
query_movimentacao = Movimentacao.objects.filter(valor_pago__isnull=False)
query_movimentacao = query_movimentacao.filter(data_lancamento__year=mes_ano.year, data_lancamento__month=mes_ano.month)
query_movimentacao.aggregate(Sum("valor_pago"))["valor_pago__sum"]

Decimal('2139.70000000000')

In [12]:
from datetime import datetime, timedelta

def mes_passado(data):
    dia = data.day
    data = data.replace(day=1) - timedelta(days=1)
    data = data.replace(day=dia)
    return data

mes_ano = datetime.strptime("2020-01", '%Y-%m')
mes_passado(mes_ano), mes_ano

(datetime.datetime(2019, 12, 1, 0, 0), datetime.datetime(2020, 1, 1, 0, 0))

In [13]:
from serase_relatorio.utils import calcula_periodo, calcula_periodo_anterior
from serase_app.models import *

from datetime import datetime, timedelta
from django.db.models import Q, F, Sum, Case, When, DecimalField, Value
from django.db.models.functions import Cast, Coalesce


usuario = User.objects.get(username="jv_eumsmo")

hoje = datetime.strptime("2020-09-10", '%Y-%m-%d')
periodo = "semanal"

data_inicio, data_fim = calcula_periodo(periodo,hoje)
data_inicio_passado, data_fim_passado = calcula_periodo_anterior(periodo, hoje)

query = Movimentacao.objects.filter(cod_usuario=usuario, valor_pago__lt=0) # Seleciona despesas do usuario
query = query.values('cod_categoria')

# Seleciona despesas do periodo atual
qperiodo_atual = query.filter(data_lancamento__gte=data_inicio, data_lancamento__lte=data_fim)
    # Soma todas as despesas de uma categoria em 'despesa_total' e ordena em ordem decrescente
qperiodo_atual = qperiodo_atual.annotate(despesa_total=Sum('valor_pago')).order_by("despesa_total")

print(data_inicio, data_fim)
print(data_inicio_passado, data_fim_passado)

query.filter(data_lancamento__gte=data_inicio_passado, data_lancamento__lte=data_fim).values('cod_categoria').annotate(
    despesa_total=Case(
        When(data_lancamento__gte=data_inicio, data_lancamento__lte=data_fim, then=Sum('valor_pago')),
        default=Value(0.0),
        output_field=DecimalField()
    ) - Case(
        When(data_lancamento__gte=data_inicio_passado, data_lancamento__lte=data_fim_passado, then=Sum('valor_pago')),
        default=Value(0.0),
        output_field=DecimalField()
    )
).aggregate(a=Sum('despesa_total'))

query.filter(data_lancamento__gte=data_inicio_passado, data_lancamento__lte=data_fim).annotate(
    despesa_total=Coalesce(Sum('valor_pago', filter=Q(data_lancamento__gte=data_inicio, data_lancamento__lte=data_fim)), 0.0) - Coalesce(Sum('valor_pago', filter=Q(data_lancamento__gte=data_inicio_passado, data_lancamento__lte=data_fim_passado)),0.0)
)

# Seleciona despesas entre o começo do ultimo periodo e o final do periodo atual
qrelacao_ultimo = query.filter(data_lancamento__gte=data_inicio_passado, data_lancamento__lte=data_fim)
    
    #
qrelacao_ultimo = qrelacao_ultimo.annotate(
    despesa_total= Coalesce(Sum('valor_pago', filter=Q(data_lancamento__gte=data_inicio, data_lancamento__lte=data_fim)), 0.0) 
    - Coalesce(Sum('valor_pago', filter=Q(data_lancamento__gte=data_inicio_passado, data_lancamento__lte=data_fim_passado)),0.0)
)

qrelacao_ultimo.order_by('despesa_total').last()
#qperiodo_antes.union(qperiodo_atual.annotate(despesa_total=-1*F('despesa_total'))).values('cod_categoria','despesa_total').annotate(diferenca=Sum('despesa_total'))
#qperiodo_atual, qperiodo_antes

2020-09-06 00:00:00 2020-09-12 00:00:00
2020-08-30 00:00:00 2020-09-05 00:00:00


{'cod_categoria': 4, 'despesa_total': Decimal('363')}

In [14]:
qperiodo_atual

<QuerySet [{'cod_categoria': 8, 'despesa_total': Decimal('-354.400000000000')}, {'cod_categoria': 10, 'despesa_total': Decimal('-206.300000000000')}, {'cod_categoria': 4, 'despesa_total': Decimal('-26.9000000000000')}]>

In [15]:
from serase_relatorio.utils import calcula_periodo, calcula_periodo_anterior
from serase_app.models import *

from datetime import datetime, timedelta
from django.db.models import Q, F, Sum,  DecimalField, Value, Count
from django.db.models.functions import Cast, Coalesce


hoje = datetime.strptime("2020-09-10", '%Y-%m-%d')
periodo = "ano"
data_inicio, data_fim = calcula_periodo(periodo, hoje)

query = Movimentacao.objects.filter(cod_usuario=usuario, data_lancamento__gte=data_inicio, data_lancamento__lte=data_fim)
query = query.filter(valor_pago__isnull=False)

total_despesas = query.count()

query = query.values(nome=F("cod_categoria__nome")).annotate(porcentagem=Cast(100.0 * Count("cod_categoria")/float(total_despesas), DecimalField(decimal_places=2)))
list(query)

ValueError: Cannot use None as a query value

In [ ]:
from serase_relatorio.utils import calcula_periodo, calcula_periodo_anterior
from serase_app.models import *

from datetime import datetime, timedelta
from django.db.models import Q, F, Case, When, Value, Count, CharField, DecimalField
from django.db.models.functions import Cast, Coalesce, Extract


hoje = datetime.strptime("2020-09-10", '%Y-%m-%d')
periodo = "anual"
data_inicio, data_fim = calcula_periodo(periodo, hoje)

query = Movimentacao.objects.filter(cod_usuario=usuario, data_lancamento__gte=data_inicio, data_lancamento__lte=data_fim)
query = query.filter(cod_padrao__isnull=False, valor_pago__isnull=False, valor_pago__lt=0)

query = query.annotate(tipo=Case(
    When(cod_padrao__valor__isnull=True, then=Value("Variáveis")),
    default=Value("Fixas"),
    output_field=CharField()
))

total = query.count()

query.values("tipo").annotate(porcentagem=Cast(100.0 * Count("tipo")/float(total), DecimalField(decimal_places=2)))

In [ ]:
from serase_relatorio.utils import calcula_periodo, calcula_periodo_anterior
from serase_app.models import *

from datetime import datetime, timedelta
from django.db.models import Q, F, Case, When, Value, Count, CharField, Sum
from django.db.models.functions import Cast, Coalesce, Extract

usuario = User.objects.get(username="jv_eumsmo")

hoje = datetime.strptime("2020-09-10", '%Y-%m-%d')
periodo = "anual"
data_inicio, data_fim = calcula_periodo(periodo, hoje)

query = Movimentacao.objects.filter(cod_usuario=usuario, data_lancamento__gte=data_inicio, data_lancamento__lte=data_fim)
query = query.filter(cod_padrao__isnull=False, valor_pago__isnull=False, valor_pago__lt=0, cod_padrao__valor__isnull=True)


query = query.annotate(mes=Extract("data_lancamento", "month")).values("mes", nome=F("cod_padrao__descricao")).annotate(valor=Sum("valor_pago"))

resultado = dict()
query = list(query)

for obj in query:
    valor = {"mes": obj["mes"], "valor": round(float(obj["valor"]), 2)}
    
    if obj["nome"] not in resultado:
        resultado[obj["nome"]] = list()
    resultado[obj["nome"]].append(valor)

resultado

In [ ]:
hoje = datetime.strptime("2020-09-10", '%Y-%m-%d')
periodo = "anual"

data_inicio, data_fim = calcula_periodo(periodo,hoje)
data_inicio_passado, data_fim_passado = calcula_periodo_anterior(periodo, hoje)

data_inicio, data_fim, data_inicio_passado, data_fim_passado

In [ ]:
Categoria.objects.get(id=8)

In [10]:
from django.db.models import Q, F, Case, When, Value, Count, CharField, Sum
from django.db.models.functions import Cast, Coalesce, Extract

Movimentacao.objects.filter(data_lancamento__isnull=False).annotate(mes=Extract("data_lancamento", "month"), ano=Extract("data_lancamento", "year")).values("mes","ano").distinct()

<QuerySet [{'mes': 11, 'ano': 2020}, {'mes': 10, 'ano': 2020}, {'mes': 9, 'ano': 2020}, {'mes': 8, 'ano': 2020}, {'mes': 7, 'ano': 2020}, {'mes': 1, 'ano': 2021}]>

In [ ]:
hoje = datetime.strptime("2020-09-10", '%Y-%m-%d')
usuario = User.objects.get(username="jv_eumsmo")
categ = Categoria.objects.first()

Movimentacao.objects.create(descricao="mov_tests", valor_pago=1.2, data_lancamento=hoje, cod_usuario=usuario, cod_categoria=categ)

In [14]:
Movimentacao.objects.filter(cod_usuario__username="jv_eumsmo", data_lancamento__lt='2020-11-01').aggregate(Sum('valor_pago'))

{'valor_pago__sum': Decimal('6215.54000000000')}

In [11]:
query = Movimentacao.objects.filter(cod_usuario__username="jv_eumsmo", cod_padrao__isnull=False)
query

<QuerySet [<Movimentacao: Spotify>, <Movimentacao: Bolsa ic>, <Movimentacao: Conta de luz>, <Movimentacao: Internet>, <Movimentacao: Sálario>, <Movimentacao: Spotify>, <Movimentacao: Bolsa ic>, <Movimentacao: Conta de luz>, <Movimentacao: Internet>, <Movimentacao: Internet>, <Movimentacao: Sálario>, <Movimentacao: Spotify>, <Movimentacao: Bolsa ic>, <Movimentacao: Conta de luz>, <Movimentacao: Internet>, <Movimentacao: Sálario>, <Movimentacao: Spotify>, <Movimentacao: Bolsa Ic>, <Movimentacao: Conta de luz>, <Movimentacao: internet>, '...(remaining elements truncated)...']>

In [12]:
query.filter(data_lancamento__isnull=True)

<QuerySet [<Movimentacao: Spotify>]>

In [31]:
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta


def add_business_days(from_date, number_of_days):
    to_date = from_date
    while number_of_days:
       to_date += timedelta(1)
       if to_date.weekday() < 5: # i.e. is not saturday or sunday
           number_of_days -= 1
    return to_date

def correct_weekday(date):
    # Data começando no domingo como 0
    return date.isoweekday()%7

def week_num(date):
    return date.isocalendar()[1]

def business_days_in_month(mes, dias):
    data = mes.replace(day=1) - timedelta(days=1)
    return add_business_days(data, dias)

def day_of_week(semana,day):
    # 1=domingo, 2=segunda, ..., 7=sabado
    data = semana + timedelta(days=correct_weekday(semana)-1 + day)
    return data

def month_of_year(ano, mes):
    data = ano.replace(day=1,month=mes)
    return data


def create_cobranca(padrao, data):
    if padrao.periodo == "mensal":
        data_geracao = business_days_in_month(data, padrao.dia_cobranca)
    elif padrao.periodo == "semanal":
        data_geracao = day_of_week(data, padrao.dia_cobranca)
    elif padrao.periodo == "anual":
        data_geracao = month_of_year(data, padrao.dia_cobranca)
    
    cobranca = Movimentacao(descricao=padrao.descricao, valor_esperado=padrao.valor, data_geracao=data_geracao, cod_usuario=padrao.cod_usuario, cod_categoria=padrao.cod_categoria, cod_padrao=padrao)
    return cobranca

padrao = PadraoMovimentacao.objects.get(descricao="teste anual")
data_ultima_cobranca = padrao.ultima_cobranca.data_geracao if padrao.ultima_cobranca else padrao.data_inicio or date.today()


print(padrao)
print(padrao.periodo)
print(padrao.ultima_cobranca, data_ultima_cobranca)

def week_num(date):
    return date.isocalendar()[1]


hoje = date.today()

cobrancas_criadas = []

if padrao.periodo == "anual" and data_ultima_cobranca.year < hoje.year:
    print("Cobrança gerada por padrão anual não esta disponivel no atual momento!")
elif padrao.periodo == "mensal" and data_ultima_cobranca < hoje: # Cobrança mensal e ultima cobrança não foi nesse mes
    # Código ainda só considera cobranças de padrão normal, ignorando as de divida.
    while data_ultima_cobranca < hoje:
        data_ultima_cobranca = data_ultima_cobranca + relativedelta(months=1)
        data_ultima_cobranca = data_ultima_cobranca.replace(day=1)
        aux = create_cobranca(padrao, data_ultima_cobranca)
        data_ultima_cobranca = aux.data_geracao
        
        if data_ultima_cobranca <= hoje:
            cobrancas_criadas.append(aux)
elif padrao.periodo == "semanal" and data_ultima_cobranca.year <= hoje.year and week_num(data_ultima_cobranca) < week_num(hoje):
    # Código ainda só considera cobranças de padrão normal, ignorando as de divida.
    while data_ultima_cobranca < hoje:
        data_ultima_cobranca = data_ultima_cobranca + timedelta(weeks=1)
        data_ultima_cobranca = data_ultima_cobranca - timedelta(days=correct_weekday(data_ultima_cobranca))
        aux = create_cobranca(padrao, data_ultima_cobranca)
        data_ultima_cobranca = aux.data_geracao
        
            
        if data_ultima_cobranca <= hoje:
            cobrancas_criadas.append(aux)

print(cobrancas_criadas)
for c in cobrancas_criadas:
    print(c, c.data_geracao)


teste anual
anual
None 2021-03-15
[]


In [5]:
from serase_app.views import *
gerar_cobranca(PadraoMovimentacao.objects.first())

[<Movimentacao: Spotify>, <Movimentacao: Spotify>]

In [4]:
padrao = PadraoMovimentacao.objects.get(descricao="Internet")
data_ultima_cobranca = padrao.ultima_cobranca.data_geracao
Movimentacao.objects.filter(cod_padrao=padrao).values("data_geracao")

<QuerySet [{'data_geracao': datetime.date(2020, 10, 7)}, {'data_geracao': datetime.date(2020, 9, 7)}, {'data_geracao': datetime.date(2020, 9, 7)}, {'data_geracao': datetime.date(2020, 8, 7)}, {'data_geracao': datetime.date(2021, 1, 7)}]>

In [3]:
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta

#UTILS

def add_business_days(from_date, number_of_days):
    to_date = from_date
    while number_of_days:
       to_date += timedelta(1)
       if to_date.weekday() < 5: # i.e. is not saturday or sunday
           number_of_days -= 1
    return to_date

def week_num(date):
    return date.isocalendar()[1]

def correct_weekday(date):
    # Data começando no domingo como 0
    return date.isoweekday()%7

def business_days_in_month(mes, dias):
    data = mes.replace(day=1) - timedelta(days=1)
    return add_business_days(data, dias)

def day_of_week(semana,day):
    # 1=domingo, 2=segunda, ..., 7=sabado
    data = semana + timedelta(days=correct_weekday(semana)-1 + day)
    return data

def month_of_year(ano, mes):
    data = ano.replace(day=1,month=mes)
    return data

#CODE


def create_cobranca(padrao, data, create=False):
    if padrao.periodo == "mensal":
        data_geracao = business_days_in_month(data, padrao.dia_cobranca)
    elif padrao.periodo == "semanal":
        data_geracao = day_of_week(data, padrao.dia_cobranca)
    elif padrao.periodo == "anual":
        data_geracao = month_of_year(data, padrao.dia_cobranca)
    
    cobranca = Movimentacao(descricao=padrao.descricao, valor_esperado=padrao.valor, data_geracao=data_geracao, cod_usuario=padrao.cod_usuario, cod_categoria=padrao.cod_categoria, cod_padrao=padrao)
    
    if create:
        cobranca.save()
    else:
        return cobranca

def gerar_cobranca(padrao, create=False):
    ultima_cobranca = padrao.ultima_cobranca
    data_ultima_cobranca = ultima_cobranca.data_geracao if ultima_cobranca else padrao.data_geracao
    hoje = date.today()

    # Vetor que armazenará novas cobranças caso sejam feitas
    cobrancas_criadas = []
    print(padrao, padrao.periodo, data_ultima_cobranca)

    # Checa periodo do padrão e se há novas cobranças a serem feitas
    if padrao.periodo == "anual" and data_ultima_cobranca.year < hoje.year:
        # Código ainda só considera cobranças de padrão normal, ignorando as de divida.
        while data_ultima_cobranca < hoje:
            data_ultima_cobranca = data_ultima_cobranca + relativedelta(years=1)
            data_ultima_cobranca = data_ultima_cobranca.replace(day=1)
            aux = create_cobranca(padrao, data_ultima_cobranca)
            data_ultima_cobranca = aux.data_geracao
            
            if data_ultima_cobranca <= hoje:
                cobrancas_criadas.append(aux)
                
    elif padrao.periodo == "mensal" and data_ultima_cobranca < hoje:
        # Código ainda só considera cobranças de padrão normal, ignorando as de divida.
        while data_ultima_cobranca < hoje:
            data_ultima_cobranca = data_ultima_cobranca + relativedelta(months=1)
            data_ultima_cobranca = data_ultima_cobranca.replace(day=1)
            aux = create_cobranca(padrao, data_ultima_cobranca)
            data_ultima_cobranca = aux.data_geracao
            
            if data_ultima_cobranca <= hoje:
                cobrancas_criadas.append(aux)
                
    elif padrao.periodo == "semanal" and data_ultima_cobranca.year <= hoje.year and week_num(data_ultima_cobranca) < week_num(hoje):
        # Código ainda só considera cobranças de padrão normal, ignorando as de divida.
        while data_ultima_cobranca < hoje:
            data_ultima_cobranca = data_ultima_cobranca + timedelta(weeks=1)
            data_ultima_cobranca = data_ultima_cobranca - timedelta(days=correct_weekday(data_ultima_cobranca))
            aux = create_cobranca(padrao, data_ultima_cobranca)
            data_ultima_cobranca = aux.data_geracao
            
                
            if data_ultima_cobranca <= hoje:
                cobrancas_criadas.append(aux)

    if create:
        return Movimentacao.objects.bulk_create(cobrancas_criadas) or []
    else:
        return cobrancas_criadas
    
def gera_cobrancas_pendentes(user, create=True):
    padroes = PadraoMovimentacao.objects.filter(cod_usuario=user)
    criadas = []
    for padrao in padroes:
        criadas = criadas + gerar_cobranca(padrao, False)
    
    if create:
        return Movimentacao.objects.bulk_create(cobrancas_criadas) or []
    else:
        return criadas
    

    


gera_cobrancas_pendentes(User.objects.get(username="jv_eumsmo"), False)

Spotify mensal 2021-01-17
Bolsa IC mensal 2020-09-08
Conta de luz mensal 2021-01-07
Internet mensal 2021-01-07
Sálario semanal 2021-01-08
teste anual anual 2018-03-01


[<Movimentacao: Spotify>,
 <Movimentacao: Spotify>,
 <Movimentacao: Bolsa IC>,
 <Movimentacao: Bolsa IC>,
 <Movimentacao: Bolsa IC>,
 <Movimentacao: Bolsa IC>,
 <Movimentacao: Bolsa IC>,
 <Movimentacao: Bolsa IC>,
 <Movimentacao: Conta de luz>,
 <Movimentacao: Conta de luz>,
 <Movimentacao: Internet>,
 <Movimentacao: Internet>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: teste anual>,
 <Movimentacao: teste anual>]

In [31]:
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta, date
from dateutil.rrule import rrule, MONTHLY, YEARLY, WEEKLY

padrao = PadraoMovimentacao.objects.get(descricao="teste anual")

days = rrule(YEARLY,dtstart=padrao.data_geracao, until=date.today(), bymonth=padrao.dia_cobranca, bymonthday=2)
for day in days:
    print(day)
date.today() if not padrao.data_fim else min(date.today(), padrao.data_fim)

2018-04-02 00:00:00
2019-04-02 00:00:00
2020-04-02 00:00:00


datetime.date(2021, 3, 21)

In [109]:
from dateutil.rrule import *

def calc_weekday(day):
    return (day+5)%7

def gerar_cobranca(padrao, create=False):
    ultima_cobranca = padrao.ultima_cobranca
    data_inicio = ultima_cobranca.data_geracao + timedelta(days=1) if ultima_cobranca else padrao.data_geracao
    data_fim = date.today() if not padrao.data_fim else min(date.today(), padrao.data_fim) 

    dias = []
    cobrancas_criadas = []

    if padrao.periodo == "anual" and data_inicio.year < data_fim.year:
        dias = rrule(YEARLY,dtstart=data_inicio, until=data_fim, bymonth=padrao.dia_cobranca, bymonthday=1)
    elif padrao.periodo == "mensal" and data_inicio < data_fim:
        dias = rrule(MONTHLY,dtstart=data_inicio, until=data_fim, bysetpos=padrao.dia_cobranca, byweekday=(MO,TU,WE,TH,FR))
    elif padrao.periodo == "semanal":
        dias = rrule(WEEKLY,dtstart=data_inicio, until=data_fim, wkst=MO, byweekday=calc_weekday(5))
    else:
        return []
    
    for dia in dias:
        mov = Movimentacao(descricao=padrao.descricao, valor_esperado=padrao.valor, data_geracao=dia, cod_usuario=padrao.cod_usuario, cod_categoria=padrao.cod_categoria, cod_padrao=padrao)
        cobrancas_criadas.append(mov)
    
    if create:
        return Movimentacao.objects.bulk_create(cobrancas_criadas) or []
    else:
        return cobrancas_criadas
    
def gera_cobrancas_pendentes(user, create=True):
    padroes = PadraoMovimentacao.objects.filter(cod_usuario=user)
    criadas = []
    for padrao in padroes:
        cob = gerar_cobranca(padrao, False)
        print(padrao, padrao.ultima_cobranca.data_geracao if padrao.ultima_cobranca else padrao.data_geracao, padrao.periodo)
        for c in cob:
            print(c.data_geracao)
        criadas = criadas + cob
    
    if create:
        return Movimentacao.objects.bulk_create(cobrancas_criadas) or []
    else:
        return criadas
    

    


gera_cobrancas_pendentes(User.objects.get(username="jv_eumsmo"), False)

Spotify 2021-01-17 mensal
2021-02-01 00:00:00
2021-03-01 00:00:00
Bolsa IC 2020-09-08 mensal
2020-10-08 00:00:00
2020-11-09 00:00:00
2020-12-08 00:00:00
2021-01-08 00:00:00
2021-02-08 00:00:00
Conta de luz 2021-01-07 mensal
2021-01-08 00:00:00
2021-02-08 00:00:00
2021-03-08 00:00:00
Internet 2021-01-07 mensal
2021-02-05 00:00:00
2021-03-05 00:00:00
Sálario 2021-01-08 semanal
2021-01-14 00:00:00
2021-01-21 00:00:00
2021-01-28 00:00:00
2021-02-04 00:00:00
2021-02-11 00:00:00
2021-02-18 00:00:00
2021-02-25 00:00:00
2021-03-04 00:00:00
2021-03-11 00:00:00
2021-03-18 00:00:00
teste anual 2018-03-01 anual
2018-04-01 00:00:00
2019-04-01 00:00:00
2020-04-01 00:00:00


[<Movimentacao: Spotify>,
 <Movimentacao: Spotify>,
 <Movimentacao: Bolsa IC>,
 <Movimentacao: Bolsa IC>,
 <Movimentacao: Bolsa IC>,
 <Movimentacao: Bolsa IC>,
 <Movimentacao: Bolsa IC>,
 <Movimentacao: Conta de luz>,
 <Movimentacao: Conta de luz>,
 <Movimentacao: Conta de luz>,
 <Movimentacao: Internet>,
 <Movimentacao: Internet>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: Sálario>,
 <Movimentacao: teste anual>,
 <Movimentacao: teste anual>,
 <Movimentacao: teste anual>]

In [9]:
Movimentacao.objects.get(id=2, cod_usuario=User.objects.last())

DoesNotExist: Movimentacao matching query does not exist.

In [20]:
from serase_app.serializers import *
a = ParametroMovimentacaoSerializer()


ParametroMovimentacaoSerializer():
    descricao = CharField(allow_blank=True, allow_null=True, max_length=40, required=False)
    valor_pago = DecimalField(allow_null=True, decimal_places=2, max_digits=9, required=False)
    data_lancamento = DateField(allow_null=True, required=False)
    categoria = CharField(max_length=20)

In [3]:
#Migrando dados para próxima versão
{
    "categoria": list(Categoria.objects.values()),
    "padrao": list(PadraoMovimentacao.objects.values()),
    "divida": list(Divida.objects.values()),
    "movimentacao": list(Movimentacao.objects.values()),
}

{'categoria': [], 'padrao': [], 'divida': [], 'movimentacao': []}

In [9]:
dic = {'categoria': [{'id': 4, 'nome': 'Lazer'},
  {'id': 5, 'nome': 'Alimentação'},
  {'id': 6, 'nome': 'Outros'},
  {'id': 7, 'nome': 'Transporte'},
  {'id': 8, 'nome': 'Compras'},
  {'id': 9, 'nome': 'Salário'},
  {'id': 10, 'nome': 'Habitação'},
  {'id': 11, 'nome': 'Saúde'},
  {'id': 12, 'nome': 'Educação'},
  {'id': 14, 'nome': 'Presente'},
  {'id': 15, 'nome': 'Venda'}],
 'padrao': [{'id': 1,
   'receita_despesa': 'despesa',
   'descricao': 'Spotify',
   'periodo': 'mensal',
   'valor': Decimal('-26.90'),
   'dia_cobranca': 1,
   'data_geracao': datetime.date(2018, 4, 18),
   'data_fim': None,
   'cod_usuario_id': 1,
   'cod_categoria_id': 4},
  {'id': 2,
   'receita_despesa': 'receita',
   'descricao': 'Bolsa IC',
   'periodo': 'mensal',
   'valor': Decimal('300.00'),
   'dia_cobranca': 6,
   'data_geracao': datetime.date(2020, 7, 8),
   'data_fim': datetime.date(2021, 7, 8),
   'cod_usuario_id': 1,
   'cod_categoria_id': 9},
  {'id': 3,
   'receita_despesa': 'despesa',
   'descricao': 'Conta de luz',
   'periodo': 'mensal',
   'valor': None,
   'dia_cobranca': 6,
   'data_geracao': datetime.date(2020, 3, 1),
   'data_fim': None,
   'cod_usuario_id': 1,
   'cod_categoria_id': 10},
  {'id': 4,
   'receita_despesa': 'despesa',
   'descricao': 'Internet',
   'periodo': 'mensal',
   'valor': Decimal('-59.90'),
   'dia_cobranca': 5,
   'data_geracao': datetime.date(2020, 2, 4),
   'data_fim': None,
   'cod_usuario_id': 1,
   'cod_categoria_id': 10},
  {'id': 5,
   'receita_despesa': 'receita',
   'descricao': 'Sálario',
   'periodo': 'semanal',
   'valor': Decimal('2356.00'),
   'dia_cobranca': 5,
   'data_geracao': datetime.date(2021, 1, 1),
   'data_fim': datetime.date(2021, 4, 12),
   'cod_usuario_id': 1,
   'cod_categoria_id': 9},
  {'id': 6,
   'receita_despesa': 'receita',
   'descricao': 'spotify',
   'periodo': 'mensal',
   'valor': Decimal('1500.00'),
   'dia_cobranca': 5,
   'data_geracao': datetime.date(2018, 4, 18),
   'data_fim': None,
   'cod_usuario_id': 1,
   'cod_categoria_id': 14},
  {'id': 7,
   'receita_despesa': 'receita',
   'descricao': 'teste anual',
   'periodo': 'anual',
   'valor': Decimal('5.00'),
   'dia_cobranca': 3,
   'data_geracao': datetime.date(2018, 3, 1),
   'data_fim': datetime.date(2021, 3, 17),
   'cod_usuario_id': 1,
   'cod_categoria_id': 15},
  {'id': 8,
   'receita_despesa': 'divida',
   'descricao': 'Credor nome',
   'periodo': 'mensal',
   'valor': None,
   'dia_cobranca': 13,
   'data_geracao': datetime.date(2021, 1, 17),
   'data_fim': datetime.date(2021, 4, 25),
   'cod_usuario_id': 1,
   'cod_categoria_id': 4}],
 'divida': [{'id': 1,
   'credor': 'Credor nome',
   'valor_pago': Decimal('0.00'),
   'valor_divida': Decimal('40.00'),
   'juros': Decimal('0.00'),
   'juros_tipo': 'simples',
   'juros_ativos': False,
   'cod_padrao_id': 8}],
 'movimentacao': [
  {'id': 3,
   'descricao': 'Spotify',
   'valor_esperado': Decimal('-26.90'),
   'valor_pago': None,
   'data_geracao': datetime.date(2020, 11, 2),
   'data_lancamento': None,
   'cod_usuario_id': 1,
   'cod_categoria_id': 4,
   'cod_padrao_id': 1},
  {'id': 4,
   'descricao': 'bebida Halloween',
   'valor_esperado': None,
   'valor_pago': Decimal('-24.00'),
   'data_geracao': None,
   'data_lancamento': datetime.date(2020, 10, 30),
   'cod_usuario_id': 1,
   'cod_categoria_id': 4,
   'cod_padrao_id': None},
  {'id': 5,
   'descricao': 'Make pro Halloween',
   'valor_esperado': None,
   'valor_pago': Decimal('-15.75'),
   'data_geracao': None,
   'data_lancamento': datetime.date(2020, 10, 30),
   'cod_usuario_id': 1,
   'cod_categoria_id': 5,
   'cod_padrao_id': None},
  {'id': 6,
   'descricao': 'Dinheiro do Spotify',
   'valor_esperado': None,
   'valor_pago': Decimal('20.00'),
   'data_geracao': None,
   'data_lancamento': datetime.date(2020, 10, 26),
   'cod_usuario_id': 1,
   'cod_categoria_id': 6,
   'cod_padrao_id': None},
  {'id': 7,
   'descricao': 'Quebrando a quarentena',
   'valor_esperado': None,
   'valor_pago': Decimal('-17.00'),
   'data_geracao': None,
   'data_lancamento': datetime.date(2020, 10, 17),
   'cod_usuario_id': 1,
   'cod_categoria_id': 7,
   'cod_padrao_id': None},
  {'id': 8,
   'descricao': 'Compras',
   'valor_esperado': None,
   'valor_pago': Decimal('-425.50'),
   'data_geracao': None,
   'data_lancamento': datetime.date(2020, 10, 10),
   'cod_usuario_id': 1,
   'cod_categoria_id': 8,
   'cod_padrao_id': None},
  {'id': 9,
   'descricao': 'Bolsa ic',
   'valor_esperado': Decimal('300.00'),
   'valor_pago': Decimal('300.00'),
   'data_geracao': datetime.date(2020, 10, 8),
   'data_lancamento': datetime.date(2020, 10, 8),
   'cod_usuario_id': 1,
   'cod_categoria_id': 9,
   'cod_padrao_id': 5},
  {'id': 10,
   'descricao': 'Conta de luz',
   'valor_esperado': Decimal('-95.25'),
   'valor_pago': Decimal('-95.25'),
   'data_geracao': datetime.date(2020, 10, 7),
   'data_lancamento': datetime.date(2020, 10, 8),
   'cod_usuario_id': 1,
   'cod_categoria_id': 10,
   'cod_padrao_id': 3},
  {'id': 11,
   'descricao': 'Internet',
   'valor_esperado': Decimal('-59.90'),
   'valor_pago': Decimal('59.90'),
   'data_geracao': datetime.date(2020, 10, 7),
   'data_lancamento': datetime.date(2020, 10, 8),
   'cod_usuario_id': 1,
   'cod_categoria_id': 10,
   'cod_padrao_id': 4},
  {'id': 12,
   'descricao': 'Sálario',
   'valor_esperado': Decimal('2356.00'),
   'valor_pago': Decimal('2356.00'),
   'data_geracao': datetime.date(2020, 10, 7),
   'data_lancamento': datetime.date(2020, 10, 7),
   'cod_usuario_id': 1,
   'cod_categoria_id': 9,
   'cod_padrao_id': 5},
  {'id': 13,
   'descricao': 'Churros',
   'valor_esperado': None,
   'valor_pago': Decimal('-5.00'),
   'data_geracao': None,
   'data_lancamento': datetime.date(2020, 10, 3),
   'cod_usuario_id': 1,
   'cod_categoria_id': 5,
   'cod_padrao_id': None},
  {'id': 14,
   'descricao': 'Spotify',
   'valor_esperado': Decimal('-26.90'),
   'valor_pago': Decimal('-26.90'),
   'data_geracao': datetime.date(2020, 10, 1),
   'data_lancamento': datetime.date(2020, 10, 8),
   'cod_usuario_id': 1,
   'cod_categoria_id': 4,
   'cod_padrao_id': 1},
  {'id': 15,
   'descricao': 'Milkshake McDonnadls',
   'valor_esperado': None,
   'valor_pago': Decimal('-10.00'),
   'data_geracao': None,
   'data_lancamento': datetime.date(2020, 9, 15),
   'cod_usuario_id': 1,
   'cod_categoria_id': 5,
   'cod_padrao_id': None},
  {'id': 16,
   'descricao': 'Compras',
   'valor_esperado': None,
   'valor_pago': Decimal('-354.40'),
   'data_geracao': None,
   'data_lancamento': datetime.date(2020, 9, 12),
   'cod_usuario_id': 1,
   'cod_categoria_id': 8,
   'cod_padrao_id': None},
  {'id': 17,
   'descricao': 'Bolsa ic',
   'valor_esperado': Decimal('300.00'),
   'valor_pago': Decimal('300.00'),
   'data_geracao': datetime.date(2020, 9, 8),
   'data_lancamento': datetime.date(2020, 9, 8),
   'cod_usuario_id': 1,
   'cod_categoria_id': 9,
   'cod_padrao_id': 2},
  {'id': 18,
   'descricao': 'Conta de luz',
   'valor_esperado': Decimal('-86.50'),
   'valor_pago': Decimal('-86.50'),
   'data_geracao': datetime.date(2020, 9, 7),
   'data_lancamento': datetime.date(2020, 9, 8),
   'cod_usuario_id': 1,
   'cod_categoria_id': 10,
   'cod_padrao_id': 3},
  {'id': 19,
   'descricao': 'Internet',
   'valor_esperado': Decimal('-59.90'),
   'valor_pago': Decimal('-59.90'),
   'data_geracao': datetime.date(2020, 9, 7),
   'data_lancamento': datetime.date(2020, 9, 8),
   'cod_usuario_id': 1,
   'cod_categoria_id': 10,
   'cod_padrao_id': 4},
  {'id': 20,
   'descricao': 'Internet',
   'valor_esperado': Decimal('-59.90'),
   'valor_pago': Decimal('-59.90'),
   'data_geracao': datetime.date(2020, 9, 7),
   'data_lancamento': datetime.date(2020, 9, 8),
   'cod_usuario_id': 1,
   'cod_categoria_id': 10,
   'cod_padrao_id': 4},
  {'id': 21,
   'descricao': 'Sálario',
   'valor_esperado': Decimal('2356.00'),
   'valor_pago': Decimal('2356.00'),
   'data_geracao': datetime.date(2020, 9, 7),
   'data_lancamento': datetime.date(2020, 9, 7),
   'cod_usuario_id': 1,
   'cod_categoria_id': 9,
   'cod_padrao_id': 5},
  {'id': 22,
   'descricao': 'Viola',
   'valor_esperado': None,
   'valor_pago': Decimal('-389.90'),
   'data_geracao': None,
   'data_lancamento': datetime.date(2020, 9, 2),
   'cod_usuario_id': 1,
   'cod_categoria_id': 4,
   'cod_padrao_id': None},
  {'id': 23,
   'descricao': 'Spotify',
   'valor_esperado': Decimal('-26.90'),
   'valor_pago': Decimal('-26.90'),
   'data_geracao': datetime.date(2020, 9, 1),
   'data_lancamento': datetime.date(2020, 9, 8),
   'cod_usuario_id': 1,
   'cod_categoria_id': 4,
   'cod_padrao_id': 1},
  {'id': 24,
   'descricao': 'Bolsa ic',
   'valor_esperado': Decimal('300.00'),
   'valor_pago': Decimal('300.00'),
   'data_geracao': datetime.date(2020, 8, 10),
   'data_lancamento': datetime.date(2020, 8, 10),
   'cod_usuario_id': 1,
   'cod_categoria_id': 9,
   'cod_padrao_id': 2},
  {'id': 25,
   'descricao': 'Conta de luz',
   'valor_esperado': Decimal('-102.80'),
   'valor_pago': Decimal('-102.80'),
   'data_geracao': datetime.date(2020, 8, 7),
   'data_lancamento': datetime.date(2020, 8, 10),
   'cod_usuario_id': 1,
   'cod_categoria_id': 10,
   'cod_padrao_id': 3},
  {'id': 26,
   'descricao': 'Internet',
   'valor_esperado': Decimal('-59.90'),
   'valor_pago': Decimal('-59.90'),
   'data_geracao': datetime.date(2020, 8, 7),
   'data_lancamento': datetime.date(2020, 8, 10),
   'cod_usuario_id': 1,
   'cod_categoria_id': 10,
   'cod_padrao_id': 4},
  {'id': 27,
   'descricao': 'Sálario',
   'valor_esperado': Decimal('2356.00'),
   'valor_pago': Decimal('2356.00'),
   'data_geracao': datetime.date(2020, 8, 7),
   'data_lancamento': datetime.date(2020, 8, 7),
   'cod_usuario_id': 1,
   'cod_categoria_id': 9,
   'cod_padrao_id': 5},
  {'id': 28,
   'descricao': 'Spotify',
   'valor_esperado': Decimal('-26.90'),
   'valor_pago': Decimal('-26.90'),
   'data_geracao': datetime.date(2020, 8, 1),
   'data_lancamento': datetime.date(2020, 8, 10),
   'cod_usuario_id': 1,
   'cod_categoria_id': 4,
   'cod_padrao_id': 1},
  {'id': 29,
   'descricao': 'Adoção do Cumpa, o Cacto',
   'valor_esperado': None,
   'valor_pago': Decimal('-15.00'),
   'data_geracao': None,
   'data_lancamento': datetime.date(2020, 7, 16),
   'cod_usuario_id': 1,
   'cod_categoria_id': 6,
   'cod_padrao_id': None},
  {'id': 30,
   'descricao': 'Outer wilds',
   'valor_esperado': None,
   'valor_pago': Decimal('-30.86'),
   'data_geracao': None,
   'data_lancamento': datetime.date(2020, 7, 10),
   'cod_usuario_id': 1,
   'cod_categoria_id': 4,
   'cod_padrao_id': None},
  {'id': 31,
   'descricao': 'Bolsa Ic',
   'valor_esperado': Decimal('300.00'),
   'valor_pago': Decimal('300.00'),
   'data_geracao': datetime.date(2021, 1, 8),
   'data_lancamento': datetime.date(2021, 1, 8),
   'cod_usuario_id': 1,
   'cod_categoria_id': 9,
   'cod_padrao_id': 5},
  {'id': 32,
   'descricao': 'Conta de luz',
   'valor_esperado': Decimal('-86.50'),
   'valor_pago': Decimal('-86.50'),
   'data_geracao': datetime.date(2021, 1, 7),
   'data_lancamento': datetime.date(2021, 1, 8),
   'cod_usuario_id': 1,
   'cod_categoria_id': 10,
   'cod_padrao_id': 3},
  {'id': 33,
   'descricao': 'internet',
   'valor_esperado': Decimal('-59.90'),
   'valor_pago': Decimal('-59.90'),
   'data_geracao': datetime.date(2021, 1, 7),
   'data_lancamento': datetime.date(2021, 1, 8),
   'cod_usuario_id': 1,
   'cod_categoria_id': 10,
   'cod_padrao_id': 4},
  {'id': 34,
   'descricao': 'Sálario',
   'valor_esperado': Decimal('2356.00'),
   'valor_pago': Decimal('2356.00'),
   'data_geracao': datetime.date(2021, 1, 7),
   'data_lancamento': datetime.date(2021, 1, 7),
   'cod_usuario_id': 1,
   'cod_categoria_id': 9,
   'cod_padrao_id': 5},
  {'id': 35,
   'descricao': 'Flauta',
   'valor_esperado': None,
   'valor_pago': Decimal('400.90'),
   'data_geracao': None,
   'data_lancamento': datetime.date(2021, 1, 2),
   'cod_usuario_id': 1,
   'cod_categoria_id': 4,
   'cod_padrao_id': None},
  {'id': 36,
   'descricao': 'Spotify',
   'valor_esperado': Decimal('-26.90'),
   'valor_pago': Decimal('-20.90'),
   'data_geracao': datetime.date(2021, 1, 17),
   'data_lancamento': datetime.date(2021, 1, 17),
   'cod_usuario_id': 1,
   'cod_categoria_id': 4,
   'cod_padrao_id': 1},
  {'id': 38,
   'descricao': 'TOP',
   'valor_esperado': None,
   'valor_pago': Decimal('55.00'),
   'data_geracao': datetime.date(2021, 1, 7),
   'data_lancamento': datetime.date(2021, 1, 8),
   'cod_usuario_id': 1,
   'cod_categoria_id': 15,
   'cod_padrao_id': None}]}

In [10]:
import datetime
#dic = {'categoria': [], 'padrao': [], 'divida': [], 'movimentacao': []}

categorias = dic["categoria"]
padroes = dic["padrao"]
dividas = dic["divida"]
movimentacoes = dic["movimentacao"]
"""
holder = []
for categoria in categorias:
    aux = Categoria(**categoria)
    holder.append(aux)
holder = Categoria.objects.bulk_create(holder)
print(holder)

holder = []
for padrao in padroes:
    aux = PadraoMovimentacao(**padrao)
    holder.append(aux)
holder = PadraoMovimentacao.objects.bulk_create(holder)
print(holder)

holder = []
for divida in dividas:
    aux = Divida(**divida)
    holder.append(aux)
holder = Divida.objects.bulk_create(holder)
print(holder)
"""
holder = []
for movimentacao in movimentacoes:
    aux = Movimentacao.objects.create(**movimentacao)
    holder.append(aux)
print(holder)



AttributeError: 'NoneType' object has no attribute 'replace'

In [8]:
Movimentacao.objects.all()

<QuerySet [<Movimentacao: Mate coro e sanduba>]>